<a href="https://colab.research.google.com/github/daylightzjr/daylightzjr/blob/main/%E7%94%9F%E6%88%90%E5%BC%8F%E5%AF%B9%E8%AF%9D%E6%9C%BA%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#生成式对话机器人

## step0 安装相关的库

In [5]:
!pip install transformers
!pip install datasets

##step1 导入相关的库

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM,Trainer,TrainingArguments ,DataCollatorForSeq2Seq
from datasets import Dataset

##step2 数据集的加载

In [8]:
dataset = Dataset.load_from_disk("./data")

In [9]:
dataset[0]

{'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
 'input': '',
 'instruction': '保持健康的三个提示。'}

##step3 数据集的预处理

In [10]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

BloomTokenizerFast(name_or_path='Langboat/bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [11]:
for i in range(2):
  instruction = tokenizer("\n".join(["Human: " +dataset[i]["instruction"], dataset[i]["input"]]).strip() + "\n\nAssistant:")
instruction

{'input_ids': [23069, 29, 210, 8254, 6744, 4744, 25703, 24676, 937, 27599, 189, 8996, 1022, 10273, 1323, 189, 189, 4122, 15263, 29], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
def process_function(examples):
  MAX_LENGTH = 256
  input_ids , attention_mask , labels = [],[],[]
  instruction = tokenizer("\n".join(["Human: " + examples["instruction"], examples["input"]]).strip() + "\n\nAssistant:")
  response = tokenizer(examples["output"]+tokenizer.eos_token)
  input_ids = instruction["input_ids"] + response["input_ids"]
  attention_mask = instruction["attention_mask"] + response["attention_mask"]
  labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
  if len(input_ids) >MAX_LENGTH:
    input_ids = input_ids[:MAX_LENGTH]
    attention_mask = attention_mask[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]
  return {
      "input_ids":input_ids,
      "attention_mask":attention_mask,
      "labels":labels
  }


In [13]:
tokenized_ds = dataset.map(process_function,remove_columns=dataset.column_names)

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

In [14]:
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [15]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant:4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [16]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

##step4 模型创建

In [17]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

##step5 配置训练器

In [18]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=2
)

##step6 创建Trainer

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
)

## step7 模型训练

In [20]:
trainer.train()

Step,Training Loss
10,3.158000
20,3.220200
30,3.109000
40,3.142800
50,3.070100
60,2.974900
70,2.933700
80,2.879600
90,2.801800
100,2.864200


TrainOutput(global_step=1678, training_loss=2.1126297597237222, metrics={'train_runtime': 5248.9641, 'train_samples_per_second': 10.234, 'train_steps_per_second': 0.32, 'total_flos': 1.1547466480730112e+16, 'train_loss': 2.1126297597237222, 'epoch': 1.9991064780342516})

##step8 模型推理

In [21]:
from transformers import pipeline
pipe = pipeline("text-generation",model="./chatbot/checkpoint-1678",tokenizer=tokenizer, device=0)

In [22]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧是指解决考试题目或解题过程的方法或步骤。考试技巧主要包括以下几点：\n\n1. 精心设计题目：根据考场上可能出现的情况和知识点，精心设计一个题目或解答，避免出现重复的题目或看似无关的步骤。\n\n2. 运用正确的逻辑推理：按照题意和相关知识，运用逻辑推理方法，避免混乱或不必要的推导。\n\n3. 运用准确的评分系统：在进行考试评估后，运用正确的评分方法，避免使用虚假或过高的评分，以获得更好的考试成绩。\n\n4. 认真复习：在考试结束至少 1 天 的时间内进行最后的复习，熟悉相关的考试要求和目的。'}]